In [ ]:
import pandas as pd
import numpy as np

In [6]:
import tensorflow as tf

# Load the Keras model
model = tf.keras.models.load_model('model_2024_hairstyle.keras')

# Convert the model to TF Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model
with open('model_2024_hairstyle.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpr96m_07m/assets


INFO:tensorflow:Assets written to: /tmp/tmpr96m_07m/assets


Saved artifact at '/tmp/tmpr96m_07m'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  128289661720656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128289661720272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128289661720080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128289661722384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128289661722576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128289661723152: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1734023253.078716  160940 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1734023253.078726  160940 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2024-12-12 20:07:33.078842: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpr96m_07m
2024-12-12 20:07:33.079188: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-12-12 20:07:33.079197: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpr96m_07m
2024-12-12 20:07:33.082569: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-12-12 20:07:33.138970: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpr96m_07m
2024-12-12 20:07:33.145071: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 66230 microseconds.


In [5]:
import tensorflow as tf

# Load the TF Lite model
interpreter = tf.lite.Interpreter(model_path='model_2024_hairstyle.tflite')

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Output index:", output_details[0]['index'])


Output index: 13


In [32]:
ls -lh model_2024_hairstyle.tflite


-rw-r--r-- 1 bench bench 77M Dec 12 20:07 model_2024_hairstyle.tflite


In [9]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [11]:
pip install pillow


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
import numpy as np
from PIL import Image

# Download and prepare the image
url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'
img = download_image(url)
img = prepare_image(img, target_size=(224, 224))

# Convert image to numpy array and normalize
image_array = np.array(img) / 255.0  # Normalize to [0, 1]


In [20]:
first_pixel_r = image_array[0, 0, 0]
print(f"First pixel R channel: {first_pixel_r}")


First pixel R channel: 0.23921568627450981


In [22]:
import tensorflow as tf

# Load the TF Lite model
interpreter = tf.lite.Interpreter(model_path='model_2024_hairstyle.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Preprocess the image to match the model input
input_data = np.expand_dims(image_array, axis=0).astype(np.float32)

# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run the model
interpreter.invoke()

# Get the output tensor
output = interpreter.get_tensor(output_details[0]['index'])
print(f"Model output: {output[0]}")


ValueError: Cannot set tensor: Dimension mismatch. Got 224 but expected 200 for dimension 1 of input 0.

In [24]:
import numpy as np
import tensorflow as tf
from PIL import Image
from io import BytesIO
import base64

# Load the model
MODEL_PATH = "/home/bench/Documents/projects/ML-zoomcamp-Homework/week9/model_2024_hairstyle.tflite"
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def preprocess_image(image_bytes):
    # Open the image
    image = Image.open(BytesIO(image_bytes))
    # Ensure RGB
    if image.mode != 'RGB':
        image = image.convert('RGB')
    # Resize to model input size
    target_size = (224, 224)
    image = image.resize(target_size, Image.NEAREST)
    # Normalize and convert to NumPy array
    image_array = np.array(image) / 255.0
    return np.expand_dims(image_array, axis=0).astype(np.float32)

def lambda_handler(event, context):
    # Get the base64-encoded image from the event
    image_data = event['body']
    image_bytes = base64.b64decode(image_data)

    # Preprocess the image
    input_data = preprocess_image(image_bytes)

    # Perform inference
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])

    # Return the result
    return {
        "statusCode": 200,
        "body": {
            "prediction": float(output[0])
        }
    }


In [29]:
from lambda_function import lambda_handler


ModuleNotFoundError: No module named 'lambda_function'

In [ ]:
from lambda_function import lambda_handler

# Read an image and encode it in base64
with open("sample_image.jpeg", "rb") as img_file:
    base64_image = base64.b64encode(img_file.read()).decode("utf-8")

# Create a mock event
event = {
    "body": base64_image
}

# Test the handler
response = lambda_handler(event, None)
print(response)


ModuleNotFoundError: No module named 'lambda_function'